#### the scope of this notebook is to extract the ROIs from the parcellation 

# IMPORTANT: in order to run on the jupyter lab, we tried to use nltools (better RAM), but their function seems to average across all voxels within an ROI as opposed to getting us the voxel-wise approach

In [13]:
import numpy as np
import scipy as sp
import pandas as pd
from matplotlib import pyplot as plt
import sys 
import os
import nibabel as nib
from nilearn import plotting
import glob
from nilearn.image import resample_to_img
from nilearn.masking import apply_mask


In [14]:
data_dir = '/home/jovyan/shared/Paranoia/fmriprep/'
mask_dir = '/home/jovyan/paranoia_project/nh-paranoia/code/data/'


In [15]:
list_of_subs = os.listdir(data_dir)
idx = list_of_subs.index('sub-tb2994')
list_of_subs = [f for f in list_of_subs if '.' not in f and f !='logs']

In [16]:
file = glob.glob(os.path.join(data_dir,'*', 'func', 'sub-tb2994*denoise_smooth6mm_task-story_run-1*nii.gz'))
file = file[0]
file



'/home/jovyan/shared/Paranoia/fmriprep/sub-tb2994/func/sub-tb2994_denoise_smooth6mm_task-story_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'

In [17]:
atlas_fname = (mask_dir + '_masks/'+ 'Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm.nii.gz')
atlas = nib.load(atlas_fname)


img = nib.load(file) # use NiBabel to load the .nii.gz file
mask = resample_to_img(atlas, img, interpolation='nearest')


In [ ]:

# node = 20
# plotting.plot_roi(mask)
# np_img = img.get_fdata()
# np_mask = mask.get_fdata()
# this_node = np_mask==node
# print(np_img[this_node].shape)



In [ ]:
# %%time
# node = 20

# # In the line below, we reach into "mask" to return its data as a simple numpy array using the method .get_fdata()
# # the ==node part returns a boolean that has True in every voxel in the node, and False everywhere else
# np_mask = mask.get_fdata()==node 
# print("Node {} contains {} voxels".format(node, np_mask.sum()))

# # We can then convert this np array back into a new Nifti image that will be just this node's mask
# # have to do .astype(int) to convert from True/False to 1/0, which is the data type we need for creating a Nifti
# this_node = nib.Nifti1Image(np_mask.astype(int), mask.affine) 

# plotting.plot_roi(this_node)
# plotting.plot_img(this_node, colorbar=True) # to verify that it is in fact 0s and 1s

In [18]:
label_node = pd.read_csv(mask_dir + '_masks/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_1mm.Centroid_RAS.csv')
label_node.head()



,ROI Label,ROI Name,R,A,S
0,1,7Networks_LH_Vis_1,-26,-34,-17
1,2,7Networks_LH_Vis_2,-26,-77,-14
2,3,7Networks_LH_Vis_3,-17,-60,-7
3,4,7Networks_LH_Vis_4,-27,-95,-4
4,5,7Networks_LH_Vis_5,-5,-92,-2


In [25]:
# schaef_dict = {}
# for node in list(label_node['ROI Name']):
#     schaef_dict[(node)] = {}
#     for subs in list_of_subs:
#         schaef_dict[node][subs] = []


In [26]:
# np_mask = mask.get_fdata()
# this_node = np_mask==1
# num = 'sub-tb2994'
# fname = glob.glob(os.path.join(data_dir,'*', 'func', f'{num}*denoise_smooth6mm_task-story_run-1*nii.gz'))[0]

# img = nib.load(fname) # use NiBabel to load the .nii.gz file
# img.get_fdata()[this_node]

In [27]:
# schaef_dict = {}
# for node in list(label_node['ROI Name']):
#     schaef_dict[(node)] = {}
#     for subs in list_of_subs:
#         schaef_dict[node] = []


In [26]:
num = list_of_subs[16]

fname = glob.glob(os.path.join(data_dir,'*', 'func', f'{num}*denoise_smooth6mm_task-story_run-1*nii.gz'))[0]
img = nib.load(fname)

In [7]:

save_dir = '/home/jovyan/paranoia_project/nh-paranoia/schaeffer_dict_roi/'
for node_num,node in enumerate(list(label_node['ROI Name'])):
    node_num = node_num+1
    print(node_num)
    
    #make a new dictionary every time 
    schaef_dict = {}
    
    for num in list_of_subs[0:14]:
        #already have the np_mask from above
        np_mask = mask.get_fdata()
        this_node = np_mask==node_num
        fname = glob.glob(os.path.join(data_dir,'*', 'func', f'{num}*denoise_smooth6mm_task-story_run-1*nii.gz'))[0]

        img = nib.load(fname) # use NiBabel to load the .nii.gz file
        
        schaef_dict[num] = img.get_fdata()[this_node]
        print(img.get_fdata()[this_node].shape)
    np.save(save_dir + f'{node}_all_subs.npy',schaef_dict)



1
(794, 526)
(794, 526)
(794, 526)
(794, 526)
(794, 526)
(794, 526)
(794, 526)
(794, 526)
(794, 526)
(794, 526)
(794, 526)
(794, 526)
(794, 526)
(794, 526)
2
(1149, 526)
(1149, 526)
(1149, 526)
(1149, 526)
(1149, 526)
(1149, 526)
(1149, 526)
(1149, 526)
(1149, 526)
(1149, 526)
(1149, 526)
(1149, 526)
(1149, 526)
(1149, 526)
3
(947, 526)
(947, 526)
(947, 526)
(947, 526)
(947, 526)
(947, 526)
(947, 526)
(947, 526)
(947, 526)
(947, 526)
(947, 526)
(947, 526)
(947, 526)
(947, 526)
4
(1590, 526)
(1590, 526)
(1590, 526)
(1590, 526)
(1590, 526)
(1590, 526)
(1590, 526)
(1590, 526)
(1590, 526)
(1590, 526)
(1590, 526)
(1590, 526)
(1590, 526)
(1590, 526)
5
(946, 526)
(946, 526)
(946, 526)
(946, 526)
(946, 526)
(946, 526)
(946, 526)
(946, 526)
(946, 526)
(946, 526)
(946, 526)
(946, 526)
(946, 526)
(946, 526)
6
(1028, 526)
(1028, 526)
(1028, 526)
(1028, 526)
(1028, 526)
(1028, 526)
(1028, 526)
(1028, 526)
(1028, 526)
(1028, 526)
(1028, 526)
(1028, 526)
(1028, 526)
(1028, 526)
7
(943, 526)
(943, 526

### Add in missing subjects to the file

In [33]:
save_dir = '/home/jovyan/paranoia_project/nh-paranoia/schaeffer_dict_roi/'


for node_num,node in enumerate(list(label_node['ROI Name'])):
    schaef_dict = np.load(save_dir + f'{node}_all_subs.npy', allow_pickle=True).item()

    node_num = node_num+1
    for num in list_of_subs[14:17]:
        print(num)
        #already have the np_mask from above

        np_mask = mask.get_fdata()
        this_node = np_mask==node_num
        fname = glob.glob(os.path.join(data_dir,'*', 'func', f'{num}*denoise_smooth6mm_task-story_run-1*nii.gz'))[0]

        img = nib.load(fname) # use NiBabel to load the .nii.gz file

        schaef_dict[num] = img.get_fdata()[this_node]
        print(img.get_fdata()[this_node].shape)
    np.save(save_dir + f'{node}_all_subs.npy',schaef_dict)


sub-tb3858
(794, 526)
sub-tb3920
(794, 526)
sub-tb3929
(794, 526)


array([[ 6.14261532, -3.68059897,  1.28394091, ..., -2.9231329 ,
         3.04521489,  7.76531887],
       [ 7.41918421, -4.54207993, -0.03298885, ..., -4.2442174 ,
         2.9870069 ,  7.38865376],
       [ 7.81869745, -6.5553031 ,  0.69222724, ..., -0.62933397,
        11.95038319, -1.0332588 ],
       ...,
       [ 3.09548187, -1.70757103, -0.23746085, ..., -3.27135038,
         2.02167296,  1.39738047],
       [ 0.47811642, -3.93914247, -1.88446939, ..., -6.15041971,
         2.73008728,  0.91406089],
       [ 1.62609625, -3.31318021,  0.38251296, ..., -5.91564131,
         0.99474192,  1.60326993]])

In [8]:
print(1)

1


# The code below uses nltools which averages across voxels in an ROI: 

### The code above crashes the Jupyterhub. Alternatively going to try to use nltools below to see if it relies on less memory than nibabel does

In [26]:
!pip install nltools

  Using cached nltools-0.4.5-py2.py3-none-any.whl (3.3 MB)
  Using cached pynv-0.2-py3-none-any.whl (3.6 kB)
  Using cached deepdish-0.3.7-py2.py3-none-any.whl (37 kB)
  Using cached tables-3.7.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.9 MB)
  Using cached numexpr-2.8.3-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (379 kB)


In [27]:
from nltools.data import Brain_Data, Adjacency
from nltools.mask import expand_mask, roi_to_brain

/srv/conda/envs/notebook/lib/python3.7/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


In [ ]:
#this code is identitical to the above
atlas_fname = (mask_dir + '_masks/'+ 'Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm.nii.gz')
atlas = nib.load(atlas_fname)

mask = Brain_Data(atlas_fname)
mask_x = expand_mask(mask)

mask.plot()

In [ ]:
node = list(schaef_dict.keys())[0]
print(node)
node_array = []
num = list_of_subs[0]
fname = glob.glob(os.path.join(data_dir,'*', 'func', f'{num}*denoise_smooth6mm_task-story_run-1*nii.gz'))[0]
data = Brain_Data(fname)



In [ ]:
for node_num,node in enumerate(list(schaef_dict.keys())):
    print(node)
    node_array = []
    for num in [list_of_subs[0]]:
        fname = glob.glob(os.path.join(data_dir,'*', 'func', f'{num}*denoise_smooth6mm_task-story_run-1*nii.gz'))[0]
        data = Brain_Data(fname)
        roi = data.extract_roi(mask_x[node_num])
        schaef_dict[node][num] = roi
schaef_dict[node][num]



In [ ]:
len(schaef_dict['7Networks_LH_Vis_5']['sub-tb2994'])